# CS4248 Project
(DON'T CLICK RUN ALL! Lemmatization takes tooo long and pls don't run it again)

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# %cd "/content/drive/My Drive/CS4248 Project"
# !cd "/content/drive/My Drive/CS4248 Project"

%pip install -r requirements.txt


[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [131]:
import pandas as pd
import numpy as np
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import os
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_filename='glove.6B.100d.txt'
# If you are running for the first time, uncomment the below 2 lines
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove*.zip
glove_path = os.path.abspath(os.path.join('.', glove_filename))

word2vec_output_file = glove_filename+'.word2vec'

glove2word2vec(glove_path, word2vec_output_file)
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
/var/folders/j2/q86pqgyn7bz6wlqds1ml4h_40000gn/T/ipykernel_38104/117655612.py:27: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_path, word2vec_output_file)


In [3]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
X_train = train['text']
y_train = train['class']

In [4]:
train.head()

,class,text
0,1,"A little less than a decade ago, hockey fans w..."
1,1,The writers of the HBO series The Sopranos too...
2,1,Despite claims from the TV news outlet to offe...
3,1,After receiving 'subpar' service and experienc...
4,1,After watching his beloved Seattle Mariners pr...


In [5]:
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])
X_test = test['text']
y_test = test['class']

In [6]:
test.head()

,class,text
0,1,When so many actors seem content to churn out ...
1,1,In what football insiders are calling an unex...
2,1,In a freak accident following Game 3 of the N....
3,1,North Koreas official news agency announced to...
4,1,The former Alaska Governor Sarah Palin would b...


## Baseline Models

In [8]:
def tfidf(X_train, X_test):

  tfidf_baseline = TfidfVectorizer(ngram_range=(1,1), max_features = 10000)

  X_train_tfidf = tfidf_baseline.fit_transform(X_train)

  X_test_tfidf = tfidf_baseline.transform(X_test)

  return X_train_tfidf, X_test_tfidf

X_train_tfidf, X_test_tfidf = tfidf(X_train, X_test)

In [17]:
# Logistic Regression - baseline

def LR(X_train_tfidf, y_train, X_test_tfidf, y_test):
  print("Training start: ", datetime.datetime.now())

  LR_classifier = LogisticRegression(random_state = 42, max_iter=2000).fit(X_train_tfidf, y_train)

  y_pred_lr = LR_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_lr))
  print("Training finish: ", datetime.datetime.now())

  return y_pred_lr

y_pred_lr = LR(X_train_tfidf, y_train, X_test_tfidf, y_test)

Training start:  2024-03-27 21:19:17.799890
              precision    recall  f1-score   support

           1       0.85      0.79      0.82       750
           2       0.82      0.38      0.52       750
           3       0.57      0.83      0.67       750
           4       0.80      0.92      0.86       750

    accuracy                           0.73      3000
   macro avg       0.76      0.73      0.72      3000
weighted avg       0.76      0.73      0.72      3000

Training finish:  2024-03-27 21:19:28.288851


In [53]:
# NB - baseline

def NB(X_train_tfidf, y_train, X_test_tfidf, y_test):

  nb_classifier = ComplementNB().fit(X_train_tfidf, y_train)

  y_pred_nb = nb_classifier.predict(X_test_tfidf)

  print(classification_report(y_test, y_pred_nb))

  return y_pred_nb

y_pred_nb = NB(X_train_tfidf, y_train, X_test_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.62      0.61      0.62       750
           2       0.62      0.46      0.53       750
           3       0.62      0.93      0.74       750
           4       0.88      0.67      0.76       750

    accuracy                           0.67      3000
   macro avg       0.68      0.67      0.66      3000
weighted avg       0.68      0.67      0.66      3000



## Preprocessing

### Lemmatization with POS Tagging
(DON'T RUN AGAIN! It takes tooooo long)

In [54]:
# WORDNET LEMMATIZER (with appropriate pos tags)
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# Define function to lemmatize each word with its POS tag

# POS_TAGGER_FUNCTION : TYPE 1
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_with_pos(tokens):
  # tokenize the sentence and find the POS tag for each token
  pos_tagged = nltk.pos_tag(tokens)

  # print(pos_tagged)

  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tagged))
  # print(wordnet_tagged)
tqmd
  lemmatized_sentence = []
  for word, tag in wordnet_tagged:
      if tag is None:
          # if there is no available tag, append the token as is
          lemmatized_sentence.append(word)
      else:
          # else use the tag to lemmatize the token
          lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
  # lemmatized_sentence = " ".join(lemmatized_sentence)

  # print(lemmatized_sentence)
  return lemmatized_sentence

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:997)>


In [136]:
# Start preprocessing

from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from tqdm import tqdm
from spellchecker import SpellChecker

punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

spell = SpellChecker()

def preprocessing(data):
  data_clean = []
  for sentence in tqdm(data):

    # Tokenization
    tokens = word_tokenize(sentence)

    # Remove punctuation and number
    tokens = [w for w in tokens if (not w in punc) and (not w.isdigit())]

    # Spell check
    # Taking too long time for each sentence, not practical to be used
    # tokens = [spell.correction(w) for w in tokens]
    # tokens = [w for w in tokens if w is not None and len(w) > 0]

    # Lemmatization based on tagging
    tokens = lemmatize_with_pos(tokens)

    data_clean.append((' ').join(tokens))
  return data_clean

X_train_clean = preprocessing(X_train)
X_test_clean = preprocessing(X_test)

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [79]:
import csv
import pandas as pd

def write_to_csv(X_train_clean, X_test_clean):
    with open('raw_data/X_train_clean.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in X_train_clean:
            writer.writerow([doc])

    with open('raw_data/X_test_clean.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        for doc in X_test_clean:
            writer.writerow([doc])

# Uncomment if you want to save X_train_clean and X_test_clean
# write_to_csv(X_train_clean, X_test_clean)

In [7]:
def read_from_file():
    a_df = pd.read_csv('raw_data/X_train_clean.csv', header=None, dtype=str)
    a = a_df[0].values
    
    b_df = pd.read_csv('raw_data/X_test_clean.csv', header=None, dtype=str)
    b = b_df[0].values
    return a, b

# Uncomment if you want to read from file
a, b = read_from_file()
X_train_clean, X_test_clean = a, b

In [85]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder()
y_train_labeled = one_hot_encoder.fit_transform(np.array(y_train).reshape(-1, 1)).toarray()
y_test_labeled = one_hot_encoder.transform(np.array(y_test).reshape(-1, 1)).toarray()


In [18]:
X_train_clean_tfidf, X_test_clean_tfidf = tfidf(X_train_clean, X_test_clean)

_ = LR(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

Training start:  2024-03-27 21:19:50.334498
              precision    recall  f1-score   support

           1       0.85      0.81      0.83       750
           2       0.84      0.42      0.56       750
           3       0.59      0.83      0.69       750
           4       0.81      0.91      0.86       750

    accuracy                           0.74      3000
   macro avg       0.77      0.74      0.73      3000
weighted avg       0.77      0.74      0.73      3000

Training finish:  2024-03-27 21:19:58.324094


In [82]:
_ = NB(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

              precision    recall  f1-score   support

           1       0.60      0.61      0.61       750
           2       0.65      0.51      0.57       750
           3       0.63      0.92      0.75       750
           4       0.87      0.64      0.74       750

    accuracy                           0.67      3000
   macro avg       0.69      0.67      0.67      3000
weighted avg       0.69      0.67      0.67      3000


In [129]:
def tfidf(X_train, X_test):

  tfidf_baseline = TfidfVectorizer(ngram_range=(1,1), max_features = 10000)

  X_train_tfidf = tfidf_baseline.fit_transform(X_train)

  X_test_tfidf = tfidf_baseline.transform(X_test)

  return X_train_tfidf, X_test_tfidf

X_train_clean_tfidf, X_test_clean_tfidf = tfidf(X_train_clean, X_test_clean)

# _ = LR(X_train_clean_tfidf, y_train, X_test_clean_tfidf, y_test)

In [12]:
class Word2VecVectorizer:
  def __init__(self, model):
    print("Loading in word vectors...")
    self.word_vectors = model
    print("Finished loading in word vectors")

  def fit(self, data):
    pass

  def transform(self, data):
    # determine the dimensionality of vectors
    v = self.word_vectors.get_vector('queen')
    self.D = v.shape[0]

    X = np.zeros((len(data), self.D*3))
    n = 0
    emptycount = 0
    for sentence in tqdm(data):
      tokens = nltk.word_tokenize(sentence.lower())
      vecs = []
      m = 0
      for word in tokens:
        try:
          # throws KeyError if word not found
          vec = self.word_vectors.get_vector(word)
          vecs.append(vec)
          m += 1
        except KeyError:
          pass
      if len(vecs) > 0:
        vecs = np.array(vecs)
        X[n] = np.concatenate((vecs.mean(axis=0), vecs.min(axis=0), vecs.max(axis=0)))
      else:
        emptycount += 1
      n += 1
    print("Number of samples with no words found: %s / %s" % (emptycount, len(data)))
    return X


  def fit_transform(self, data):
    self.fit(data)
    return self.transform(data)

In [132]:
word2vec_vectorizer = Word2VecVectorizer(word2vec_model)
X_train_word2vec = word2vec_vectorizer.fit_transform(X_train)

Loading in word vectors...
Finished loading in word vectors


100%|███████████████████████████████████████████████████████████████████████| 48854/48854 [03:28<00:00, 233.84it/s]

Number of samples with no words found: 2 / 48854


In [133]:
# apply to test
X_test_word2vec = word2vec_vectorizer.fit_transform(X_test)

100%|█████████████████████████████████████████████████████████████████████████| 3000/3000 [00:12<00:00, 247.88it/s]

Number of samples with no words found: 0 / 3000


In [134]:
X_train_combined = np.concatenate((X_train_clean_tfidf.toarray(), X_train_word2vec), axis=1)
# X_train_combined =  X_train_word2vec
print(X_train_clean_tfidf.shape)
print(X_train_word2vec.shape)
print(X_train_combined.shape)

X_test_combined = np.concatenate((X_test_clean_tfidf.toarray(), X_test_word2vec), axis=1)
# X_test_combined =  X_test_word2vec

(48854, 10000)
(48854, 300)
(48854, 10300)


In [101]:
_ = LR(X_train_combined, y_train, X_test_combined, y_test)

Training start:  2024-03-21 10:05:32.712848
              precision    recall  f1-score   support

           1       0.86      0.79      0.82       750
           2       0.71      0.37      0.48       750
           3       0.53      0.70      0.60       750
           4       0.75      0.92      0.82       750

    accuracy                           0.69      3000
   macro avg       0.71      0.69      0.68      3000
weighted avg       0.71      0.69      0.68      3000

Training finish:  2024-03-21 12:40:47.537759


In [135]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam,Adagrad
import tensorflow_addons as tfa
from keras import regularizers

epochs = 5

model = Sequential()
model.add(Dense(256, activation='sigmoid', input_shape=(X_train_combined.shape[1],)))
model.add(Dropout(0.5))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(128, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=0.001),
                metrics=['accuracy', tfa.metrics.F1Score(average='macro',num_classes=4)])
model.summary()


model.fit(X_train_combined, y_train_labeled, epochs=epochs, validation_data=(X_test_combined, y_test_labeled))

y_pred = model.predict(X_test_combined)

y_pred_inverse = one_hot_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_inverse))


Model: "sequential_64"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_196 (Dense)           (None, 256)               2637056   
                                                                 
 dropout_114 (Dropout)       (None, 256)               0         
                                                                 
 dense_197 (Dense)           (None, 128)               32896     
                                                                 
 dropout_115 (Dropout)       (None, 128)               0         
                                                                 
 dense_198 (Dense)           (None, 4)                 516       
                                                                 
Total params: 2670468 (10.19 MB)
Trainable params: 2670468 (10.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
152